In [ ]:
import numpy as np
import numpy.random as random
import scipy.sparse, scipy.sparse.linalg
import matplotlib.pyplot as plt

## Prologue: Hamiltonian systems

Hamiltonian mechanics is a particular way of describing classical physical systems.

* **The players**: position $q$, momentum $p$, and the total energy $H(q, p)$ of the system
* **The rules**: Hamilton's equations of motion,
$$\begin{align}
\dot q & = +\frac{\partial H}{\partial p} \\
\dot p & = -\frac{\partial H}{\partial q}
\end{align}$$
* When $p = m\dot q$, and $H = $ kinetic energy + potential energy, Hamilton's equations of motion are equivalent to Newton's.

Some very important things:
* The energy $H$ is conserved along trajectories of the ODE.
* The volume in phase space is conserved.
Take a "blob" $D$ of position/momentum pairs, now evolve them all for a time $t$ using Hamilton's equations; this gives a morphed blob, $D_t$.
Then $\mathrm{vol}(D) = \mathrm{vol}(D_t)$.

#### Ex: coupled oscillators

The Hamiltonian:

$$H = \frac{|p|^2}{2m} + \frac{1}{2}q^*Lq$$

where $L$ is the stiffness matrix.

#### Ex: gravity

$$H = \frac{|p|^2}{2\mu} + \frac{GMm}{|q|}$$

#### The Verlet algorithm

There are many ways to integrate first-order systems of ODE.
Suppose that the Hamiltonian is separable into a kinetic energy $K$ and a potential energy $U$:

$$H = K(p) + U(q)$$

The *Verlet algorithm* updates the positions first, then uses the new position value to update the momentum:

$$\begin{align}
q_{n + \frac{1}{2}} & = q_n + \frac{\delta t}{2} \cdot \frac{\partial K}{\partial p}(p_n) \\
p_{n + 1} & = p_n - \delta t \cdot \frac{\partial U}{\partial q}(q_{n + \frac{1}{2}}) \\
q_{n + 1} & = q_{n + \frac{1}{2}} + \frac{\delta t}{2}\cdot\frac{\partial K}{\partial p}(p_{n + 1})
\end{align}$$

**Useful things:**
* The phase volume is preserved, like for Hamiltonian systems.
* The trajectory exactly preserves a perturbed Hamiltonian $H + \delta H$, where $\delta H \sim \delta t$.

In [ ]:
def kinetic_energy(p):
    return 0.5 * np.sum(p**2)

def velocity(p):
    return p

n = 128
diag = np.ones(n)
diag[0] = 0
D = scipy.sparse.diags([diag, -np.ones(n - 1)], [0, -1])
Λ = scipy.sparse.diags([np.ones(n)], [0])
L = D.T * Λ * D
def potential_energy(q):
    return 0.5 * np.dot(L * q, q)

def potential_gradient(q):
    return L * q

In [ ]:
q0 = random.RandomState().normal(size=n)
p0 = np.zeros(n)

δt = 0.01
num_steps = int(2 * np.pi / δt)

In [ ]:
q = q0.copy()
p = p0.copy()

energy = np.zeros(num_steps)

for k in range(num_steps):
    q += δt / 2 * velocity(p)
    p -= δt * potential_gradient(q)
    q += δt / 2 * velocity(p)
    
    energy[k] = kinetic_energy(p) + potential_energy(q)

In [ ]:
fig, ax = plt.subplots()
ax.plot(energy)
plt.show(fig)

In [ ]:
q = q0.copy()
p = p0.copy()

energy = np.zeros(num_steps)
for k in range(num_steps):
    qk = q.copy()
    q += δt * velocity(p)
    p -= δt * potential_gradient(qk)
    
    energy[k] = kinetic_energy(p) + potential_energy(q)

In [ ]:
fig, ax = plt.subplots()
ax.plot(energy)
plt.show(fig)

In [ ]:
q = q0.copy()
p = p0.copy()

energy = np.zeros(num_steps)
for k in range(num_steps):
    pk = p.copy()
    p = scipy.sparse.linalg.spsolve(scipy.sparse.eye(n) + δt**2 * L, pk - δt * L * q)
    q += δt * p
    
    energy[k] = kinetic_energy(p) + potential_energy(q)

In [ ]:
fig, ax = plt.subplots()
ax.plot(energy)
plt.show(fig)

## Hamiltonian Monte Carlo

MCMC simulation works with any reversible transition kernel.
The idea of HMC is to augment the state $q$ with a *pseudo-momentum* variable $p$ and use Hamiltonian dynamics to update both $q$ and $p$.